# Advent of Code: Day 12

In [55]:
using Base.Iterators
using Match
using StatsKit
using Underscores

#f = "ExampleInput.txt"
#f = "Example2Input.txt"
#f = "Example3Input.txt"
f = "SolutionInput.txt"

function parse_edge!(Edges, line)
  tokens = split(line, "-")
  v0 = tokens[1]
  v1 = tokens[2]
  if !haskey(Edges, v0)
    Edges[v0] = []
  end
  if !haskey(Edges, v1)
    Edges[v1] = []
  end
  push!(Edges[v0], v1)
  push!(Edges[v1], v0)
  Edges
end

Edges = Dict()
foreach(line -> parse_edge!(Edges, line), readlines(f))
Edges

Dict{Any, Any} with 12 entries:
  "cd"    => Any["yk"]
  "LP"    => Any["cb", "yk", "bf"]
  "my"    => Any["PK", "start", "BN", "lj"]
  "lj"    => Any["cb", "bf", "BN", "start", "my"]
  "BN"    => Any["yk", "bf", "end", "my", "lj"]
  "start" => Any["my", "PK", "lj"]
  "EP"    => Any["yk"]
  "yk"    => Any["PK", "BN", "cd", "bf", "LP", "EP"]
  "end"   => Any["bf", "cb", "BN"]
  "PK"    => Any["yk", "my", "cb", "bf", "start"]
  "bf"    => Any["end", "yk", "lj", "BN", "PK", "LP"]
  "cb"    => Any["LP", "end", "lj", "PK"]

## Part 1

In [56]:
function paths(Edges)
  found_paths = []
  next_vertices = map(vertex -> (vertex, ["start"]), Edges["start"])

  while !isempty(next_vertices)
    current_vertex, partial_path = pop!(next_vertices)

    if current_vertex == "end"
      push!(found_paths, [partial_path..., current_vertex])
      continue
    end

    seen_small_caves = @_ partial_path                     |>
                          filter(_ ∉ ["start", "end"], __) |>
                          filter(islowercase(_[1]), __)

    child_vertices = @_ Edges[current_vertex] |>
                        filter(_ ∉ ["start",seen_small_caves...], __)
    
    for child_vertex ∈ child_vertices
      push!(next_vertices, (child_vertex, [partial_path..., current_vertex]))
    end
  end

  found_paths
end 
paths(Edges) |> length

3298

## Part 2

In [57]:
function paths_2(Edges)
  found_paths = []
  next_vertices = map(vertex -> (vertex, ["start"]), Edges["start"])

  while !isempty(next_vertices)
    current_vertex, partial_path = pop!(next_vertices)

    if current_vertex == "end"
      push!(found_paths, [partial_path..., current_vertex])
      continue
    end

    seen_small_caves_with_count = @_ partial_path                     |>
                                     filter(_ ∉ ["start", "end"], __) |>
                                     filter(islowercase(_[1]), __)    |>
                                     countmap                         |>
                                     collect

    seen_small_cave_twice = @_ seen_small_caves_with_count |>
                               map(_[2]>1,__)              |>
                               any
    
    seen_small_caves = []
    if seen_small_cave_twice
      seen_small_caves = map(c -> c[1], seen_small_caves_with_count)
    end

    if current_vertex ∈ seen_small_caves
      continue
    end

    child_vertices = @_ Edges[current_vertex] |>
                        filter(_ ∉ ["start",seen_small_caves...], __)
    
    for child_vertex ∈ child_vertices
      push!(next_vertices, (child_vertex, [partial_path..., current_vertex]))
    end
  end

  found_paths
end

paths_2(Edges) |> length

93572